# Vaccination Effects On Covid19

These figures aim to show how vaccinations in the UK have helped to mitigate the spread of covid19 as well as to compare the vaccine responses in the UK between two of its major countries, **England** and **Scotland**.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make graphs larger
plt.rcParams['figure.dpi'] = 120

In [3]:
#Loads data from the json files 
with open("vaccine_effect.json", "rt") as INFILE:
    data1=json.load(INFILE)
with open("vaccination_England.json", "rt") as INFILE:
    data2=json.load(INFILE)
with open("vaccination_Scotland.json", "rt") as INFILE:
    data3=json.load(INFILE)
jsondata={1:data1, 2:data2, 3:data3}

In [4]:
# Puts the date data into the date time format. 
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# Extracts the data section from the json file
# Creates a range of days between the first and last days
# Adds the remaining data for each date. 
# Returns a table of values to be plot
def wrangle_data1(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate1=parse_date(dates[0])
    enddate1=parse_date(dates[-1])
    index=pd.date_range(startdate1, enddate1, freq='D')
    vaccine_effectdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths', 'vaccines'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths', 'vaccines']:
            if pd.isna(vaccine_effectdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                vaccine_effectdf.loc[date, column]=value
    vaccine_effectdf.fillna(0.0, inplace=True)
    return vaccine_effectdf

def wrangle_data2(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate2=parse_date(dates[0])
    enddate2=parse_date(dates[-1])
    index=pd.date_range(startdate2, enddate2, freq='D')
    vaccination_Englanddf=pd.DataFrame(index=index, columns=['vaccines1','vaccines2'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['vaccines1','vaccines2']:
            if pd.isna(vaccination_Englanddf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                vaccination_Englanddf.loc[date, column]=value
    vaccination_Englanddf.fillna(0.0, inplace=True)
    return vaccination_Englanddf

def wrangle_data3(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate3=parse_date(dates[0])
    enddate3=parse_date(dates[-1])
    index=pd.date_range(startdate3, enddate3, freq='D')
    vaccination_Scotlanddf=pd.DataFrame(index=index, columns=['vaccines1','vaccines2'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['vaccines1','vaccines2']:
            if pd.isna(vaccination_Scotlanddf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                vaccination_Scotlanddf.loc[date, column]=value
    vaccination_Scotlanddf.fillna(0.0, inplace=True)
    return vaccination_Scotlanddf

vaccine_effectdf=wrangle_data1(jsondata[1]) 
vaccination_Englanddf = wrangle_data2(jsondata[2])
vaccination_Scotlanddf = wrangle_data3(jsondata[3]) 

## Download current data

Firstly, you may click the button below to update the data from Public Health England. This allows you to obtain up-to-date graphical representations when you next load this webpage. ***After clicking this update button, please wait a few seconds***(this is needed as so not to to barred from accessing the data due to too great a request at once)

In [5]:
# Gets data from the Public Health England API for each graph.
# Stores this information in a json file
def access_api():
    from uk_covid19 import Cov19API
    import json
    import time
    
    
    filters = [
        'areaType=nation',
        'areaName=England'
    ]


    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "newDailyNsoDeathsByDeathDate",
        "vaccines": "newPeopleVaccinatedSecondDoseByVaccinationDate"  
    }
    
    api = Cov19API(filters=filters, structure=structure)
    
    vaccine_effect=api.get_json()
    
    with open("vaccine_effect.json", "wt") as OUTF:
        json.dump(vaccine_effect, OUTF)
        
    time.sleep(0.25)# tells the computer to wait 0.5 seconds before running the remainder of the code
    
    filters = [
    'areaType=nation',
    'areaName=England'
    
    ]

    structure = {
    "date": "date",
    "vaccines1": "newPeopleVaccinatedFirstDoseByVaccinationDate",  
    "vaccines2": "newPeopleVaccinatedSecondDoseByVaccinationDate"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    
    vaccination_England=api.get_json()
    
    
    with open("vaccination_England.json", "wt") as OUTF:
        json.dump(vaccination_England, OUTF)
        
    time.sleep(0.25)
    
    filters = [
    'areaType=nation',
    'areaName=Scotland'
    
    ]

    structure = {
    "date": "date",
    "vaccines1": "newPeopleVaccinatedFirstDoseByVaccinationDate",  
    "vaccines2": "newPeopleVaccinatedSecondDoseByVaccinationDate"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    
    vaccination_Scotland=api.get_json()
    
    
    with open("vaccination_Scotland.json", "wt") as OUTF:
        json.dump(vaccination_Scotland, OUTF)

    with open("vaccine_effect.json", "rt") as INFILE:
        data1=json.load(INFILE)
    
    with open("vaccination_England.json", "rt") as INFILE:
        data2=json.load(INFILE)
        
    with open("vaccination_Scotland.json", "rt") as INFILE:
        data3=json.load(INFILE)
    
    jsondata={1:data1, 2:data2, 3:data3}
    return jsondata 

In [6]:
# wrangles the newly obtained data from the API and overwrites the previous data with new data
def api_button_callback(button):
    apidata=access_api()
    global vaccine_effectdf
    global vaccination_Englanddf
    global vaccination_Scotlanddf
    
    vaccine_effectdf = wrangle_data1(apidata[1]) 
    vaccination_Englanddf = wrangle_data2(apidata[2])
    vaccination_Scotlanddf = wrangle_data3(apidata[3]) 
    
    refresh_graph1()
    refresh_graph2()
    refresh_graph3()
    
    apibutton.icon="check"
    apibutton.disabled=True

# button press to call the function to get new API data and also refresh the following graphs   
apibutton=wdg.Button(
    description='Update Data',
    disabled=False,
    button_style='success', 
    tooltip='Click button to update graphs and data to represent latest ',
    icon='download'
)

apibutton.on_click(api_button_callback)
display(apibutton)

Button(button_style='success', description='Update Data', icon='download', style=ButtonStyle(), tooltip='Click…

## Graphs and Analysis

### Vaccination Effect

This first graph shows the effect of the increasing number of vaccinations on the covid case, hospitalization and death rates. It can be seen from the graph, that as the number of vaccinations hits its peak, the number of covid cases and especially hospitalizations and deaths decrease greatly. It is only when the number of persons getting vaccinated starts to fall off ,that these negative metrics begin to rise again.

In [10]:
# creates a widget for selecting different elements of a graph
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths', 'vaccines'],
    value=['cases', 'hospital', 'deaths', 'vaccines'],
    rows=4,
    description='Statistics:',
    disabled=False
)

# creates a widget for switching graph between log and linear
scale1=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)


controls1=wdg.HBox([series, scale1])
clrplot = 0

# function to plot the graph
def VaccineEffect_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vaccine_effectdf[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click each category to see graph details")
        print("(CTRL + Click to select multiple categories)")

# function to refresh the graph with new data. Does this by simply switching from linear to log and back to linear graph to use new data.
def refresh_graph1():
    current=scale1.value
    if current==scale1.options[0]:
        other=scale1.options[1]
    else:
        other=scale1.options[0]
    scale1.value=other 
    scale1.value=current 
    

graph=wdg.interactive_output(VaccineEffect_graph, {'gcols': series, 'gscale': scale1})
display(controls1, graph)

Output()

### England vs Scotland 

The final two graphs in this webpage are used to compare the vaccination rollouts between **England** and **Scotland**. From the graphs it can be seen that between the two countries, Scotland managed to begin vaccinating the majority of its population earlier than England. The peak of vaccinations in Scotland started at around February, whilst in England only started in late March close to April.

It can also be seen that their is quite a large discrepancy between the number of persons getting their first and second doses.In England there is around a 200,000 discrepancy between the peak of those getting their first and second doses, whilst in Scotland this discrepancy is around 100,000.

In [11]:
series=wdg.SelectMultiple(
    options=['vaccines1', 'vaccines2'],
    value=['vaccines1', 'vaccines2'],
    rows=4,
    description='Statistics:',
    disabled=False
)

scale2=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)


controls2=wdg.HBox([series, scale2])
clrplot = 0

def VaccinationEngland_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vaccination_Englanddf[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click each category to see graph details")
        print("(CTRL + Click to select multiple categories)")
        

def refresh_graph2():
    current=scale2.value
    if current==scale2.options[0]:
        other=scale2.options[1]
    else:
        other=scale2.options[0]
    scale2.value=other 
    scale2.value=current 
    

graph=wdg.interactive_output(VaccinationEngland_graph, {'gcols': series, 'gscale': scale2})
display(controls2, graph)

Output()

In [12]:
series=wdg.SelectMultiple(
    options=['vaccines1', 'vaccines2'],
    value=['vaccines1', 'vaccines2'],
    rows=4,
    description='Statistics:',
    disabled=False
)

scale3=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)


controls3=wdg.HBox([series, scale3])
clrplot = 0
def VaccinationScotland_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vaccination_Scotlanddf[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click each category to see graph details")
        print("(CTRL + Click to select multiple categories)")

def refresh_graph3():
    current=scale3.value
    if current==scale3.options[0]:
        other=scale3.options[1]
    else:
        other=scale3.options[0]
    scale3.value=other 
    scale3.value=current 
        
graph=wdg.interactive_output(VaccinationScotland_graph, {'gcols': series, 'gscale': scale3})
display(controls3, graph)

Output()

**Author and Copyright Notice** Created by David Browne. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*